<a href="https://colab.research.google.com/github/wajidminhas/AI-DREAMER/blob/main/hooks/hooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 671.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.4/161.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
)
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [4]:
# Imports
import asyncio
import random
from typing import Any

from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, Usage, function_tool

## 1. Basic Example (Understand Core Concept)

In [5]:
class TestHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0
        self.name = "TestHooks"

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### {self.name} {self.event_counter}: Agent {agent.name} started. Usage: {context.usage}")

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(f"### {self.name} {self.event_counter}: Agent {agent.name} ended. Usage: {context.usage}, Output: {output}")

In [7]:
from agents import AgentHooks
class TestAgHooks(AgentHooks):
    def __init__(self, ag_display_name):
        self.event_counter = 0
        self.ag_display_name = ag_display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} started. Usage: {context.usage}")

    async def on_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(f"### {self.ag_display_name} {self.event_counter}: Agent {agent.name} ended. Usage: {context.usage}, Output: {output}")


In [10]:
start_hook = TestHooks()

start_agent = Agent(
    name="Content Moderator Agent",
    instructions="You are content moderation agent. Watch social media content received and flag queries that need help or answer. We will answer anything about AI?",
    model=model,
    hooks= TestAgHooks(ag_display_name="Content Moderator Agent")
)

async def main():
  result = await Runner.run(
      start_agent,
      hooks=start_hook,
      input=f"Will Agentic AI Die at end of 2025?."
  )

  print(result.final_output)
  # print(result.max_turn())

asyncio.run(main())
print("--end--")

### TestHooks 1: Agent Content Moderator Agent started. Usage: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)
### Content Moderator Agent 1: Agent Content Moderator Agent started. Usage: Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)


### TestHooks 2: Agent Content Moderator Agent ended. Usage: Usage(requests=1, input_tokens=41, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=290, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=331), Output: This query expresses concern about the future of "Agentic AI." While I cannot predict the future, I can offer some general information about Agentic AI and the factors that might influence its development and adoption:

Agentic AI refers to AI systems that can act autonomously to achieve specific goals. These systems often involve multiple AI models working together, and are designed to make decisions and take actions without constant human intervention.

**Factors that could influence the future of Agentic AI:**

*   **Technological advancements:** Continued progress in areas like large language models, reinforcement learning, and robotics will be crucial for the development of more capable and reliable agentic AI systems.
*  